In [1]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Define the directory containing the PDF files
pdf_directory = './data'

### KGLS 질문

### 논문을 벡터 db에 넣기

In [2]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

### 인적정보 가져오기

In [3]:
import json

class User:
    def __init__(self, info_dir):
        # Read the JSON file
        with open(info_dir, 'r') as file:
            data = json.load(file)
        self.name = data['name']
        self.phone = data['phone']
        self.age = data['age']
        self.gender = data['gender']
        self.education = data['education']
        self.merry = data['merry']
        self.children = data['children']
        self.religion = data['religion']
        self.income = data['income']
        self.economy_states = data['economy_states']
        self.health_states = data['health_states']
        
# Specify the path to the JSON file
info_dir = 'data/user_info.json'
user = User(info_dir)
print(user.name)

김영수


### KGLS 질문/답변 가져오기

In [12]:
import json

class KGLSData:
    def __init__(self, file_path):
        # JSON 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        self.kgls = data['KGLS']
        
    def get_question(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['question']
        return None
    
    def get_user_choice(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['user_choose']
        return None
    
    def get_all_questions(self):
        return [item['question'] for item in self.kgls]
    def get_all_user_choices(self):
        return [item['user_choose'] for item in self.kgls]
    def get_all_user_choices_sum(self):
        return sum([item['user_choose'] for item in self.kgls])


# 파일 경로 지정
file_path = './data/kgls_dummy_v2.json'
kgls_data = KGLSData(file_path)

# 예시 출력
print(kgls_data.get_question(1))       # "나는 가족들과 매일 대화를 나눈다."
print(kgls_data.get_user_choice(1))    # 3
print(kgls_data.get_all_questions())   # 모든 질문 리스트 출력


나는 가족들과 매일 대화를 나눈다.
3
['나는 가족들과 매일 대화를 나눈다.', '나는 가깝게 지내는 사람들이 있다.', '나는 쓸모없는 사람이라고 느껴진다.', '나를 이해해주는 사람이 있다.', '사람들은 나와 겉으로만 어울리는 것 같다.', '나는 자녀에게 고민을 이야기할 수 있다.', '나는 주변 사랑과의 관계가 만족스럽다.', '가족들은 나를 예전처럼 대해주지 않는다.', '친구나 이웃들은 나에게 관심을 둔다.', '내가 아플 때 나를 보살펴줄 가족이 있다.', '나는 의지할 친구가 있다.', '나는 온종일 할 일없이 시간을 보낸다.', '나는 사회에서 필요한 사람이다.', '나는 가족에게 의지할 수 있다.']


In [28]:
KGLS="""본 연구에서 개발된 한국 노인의 외로움 측정도구는
가족관계 외로움 5문항(1번, 6번, 8번, 10번, 14번)이며
가족관계 외로움의 총점은 4점~20점 사이입니다.
사회적 외로움 6문항(2번, 4번, 5번, 7번, 9번, 11번)이며
가족관계 외로움의 총점은 6점~24점 사이입니다.
소속감 3문항은(3번, 12번, 13번)이며
소속감에 대한 외로움의 총점은 3점~12점 사이입니다.
세 가지 하위요인을 갖는
14문항의 4점 Likert 척도로 구성되었다. 가능한 점수의 범위는 14~56점이며,
점수가 높을수록 외로움이 큼을 의미한다.
역문항은 1번, 2번, 4번, 6번, 7번, 9번, 10번, 11번, 13번, 14번 문항으로 구성되어 있습니다. """

### 프롬프트

In [29]:
#출력 양식
OUTPUT_FORMAT=f"""
### KGLS 기반 답변 분석 \\
KGLS 14문항은 likert 4점 척도로 1점~4점 사이의 값으로 사용자의 답변을 받습니다.\n \\
[KGLS 14 문항과 답변에 대한 분석]\\
- 000 님의 KGLS 점수는 00점입니다.\\
- 가족관계 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
  사용자의 가족관계 외로움의 총합은 00점 입니다. 따라서 ... 수치가 ...기에 ...하며 ...하다고 볼 수 있습니다.  \\
- 사회적 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
  사용자의 사회적 외로움의 총합은 00점 입니다. 따라서 ... 수치가 ...기에 ...하며 ...하다고 볼 수 있습니다.  \\
- 소속감에 대한 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
  사용자의 소속감에 대한 외로움의 총합은 00점 입니다 따라서 ... 수치가 ...기에 ...하며 ...하다고 볼 수 있습니다.  \\
- 14문항 중 역문항은 1번, 2번, 4번, 6번, 7번, 9번, 10번, 11번, 13번, 14번 문항으로 구성되어 있습니다.\\
  역문항의 총합은 00점이기에 ..한 결과를 나타냅니다..
[대화 시스템 프롬프트]\\
- 당신은 000님을 위한 대화 상대가 되어주어야합니다. 아래의 참고사항을 반영해 출력하세요.\\
- 대화중 사용자의 기억은 반드시 기억해야합니다.\\
- 말투는 ...하고, ...해야할 것입니다. \\
- ...\\
[분석 결과에 대한 이유]\\
- 각 문항에 대한 외로움의 분석과 설명 : ...\\
- 판단 근거 : ...   \\
- 정량적 지표 : ... \\
 """

# KGLS 14문항 점수 : {kgls_data.get_all_user_choices_sum()}점 \n\\
# {user.name}님은 {kgls_data.get_all_user_choices_sum()} 점으로 아래의 기준에 따라 ~~한 외로움을 가지고 있습니다 \n \\

In [30]:
SYS_PROMPT = f"""
    사용자의 인적정보, KGLS 질문과 한국형 외로움에 대한 연구(KGLS 논문)를 바탕으로 어떤 한국형 외로움을 가지고있는지 설명하기 위해 시스템 프롬프트 설정을 하는 시스템입니다\\
    {KGLS}은 좀 더 모델에 특화된 KGLS기반 평가 기준입니다. \\\
    ### 사용자의 외로움
    그 뒤에는 논문을 바탕으로 사용자의 한국형 외로움에 대해 해석해주세요. \\
    ### 사용자가 취약한 외로움
    사용자가 취약한 외로움에 대해서 알려주세요. 인적사항에 대해 큰 가중치를 부여해주세요. \\
    
    ###외로움 구분 점수 기준 \\
    14점 ~ 24점: 낮은 외로움 \n\\
    25점 ~ 35점: 중간 정도의 외로움 \n \\
    36점 ~ 46점: 높은 외로움\n \\
    47점 ~ 56점: 매우 높은 외로움\n \\
    사용자 인적 정보\\
    사용자의 이름은 {user.name}, 나이는 {user.age}, 성별은 {user.gender}, 학력은 {user.education}, 결혼 여부는 {user.merry}, 자녀 수는 {user.children}, 종교는 {user.religion}, 소득은 {user.income}, 경제 상태는 {user.economy_states}, 건강 상태는 {user.health_states}입니다. \\
    
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
이 사용자가 가지고 있는 한국형 외로움을 설명해주세요. \\
그리고 어떤 대화 상대가 되어주어야 하는지 출력해주세요. \\
"""

모델 선언

In [31]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''
3.외로움 파악을 위한 KGLS 객관식 14문항과 사용자가 입력한 답변={question} \\
4. 한국형 노인 외로움의 지표인 KGLS={context}\\

출력은 앞서 입력된 내용을 바탕으로 출력해야합니다. 특히 분석 결과는 3,4번 항목을 참고하여 충분히 설명해야만 합니다.
출력 예시:

'''+OUTPUT_FORMAT
     

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  f"""
Q: KGLS 답변 결과와 인적사항을 입력해주세요! \\
A: 네 알겠습니다! 양식은 다음과 같습니다
    KGLS data format\\
    {kgls_data.get_all_questions()} 이게 KGLS질문이에요.\\
    {kgls_data.get_all_user_choices()} 이게 KGLS질문에 대한 사용자의 답변이에요.\\\\
""" + INPUT_PROMPT  # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke(input_message))


> ### KGLS 기반 답변 분석
> KGLS 14문항은 Likert 4점 척도로 1점~4점 사이의 값으로 사용자의 답변을 받습니다.
> 
> [KGLS 14 문항과 답변에 대한 분석]
> - 김영수 님의 KGLS 점수는 42점입니다.
> - 가족관계 외로움은 가족과의 상호작용 부족, 정서적 지지 부족 등의 특징을 가지고 있으며, 사용자의 경우 건강 상태가 좋지 않아 가족과의 상호작용이 줄어들고, 정서적 지지를 받기 어려운 상황으로 인해 나타납니다.
>   사용자의 가족관계 외로움의 총합은 15점입니다. 따라서 이 수치가 중간 정도의 외로움에 해당하며, 가족과의 관계에서 어느 정도의 외로움을 느끼고 있다고 볼 수 있습니다.
> - 사회적 외로움은 사회적 지지 부족, 친구 및 이웃과의 상호작용 부족 등의 특징을 가지고 있으며, 사용자의 경우 경제 상태가 보통이고 건강 상태가 나빠 사회적 활동이 제한되어 나타납니다.
>   사용자의 사회적 외로움의 총합은 18점입니다. 따라서 이 수치가 중간 정도의 외로움에 해당하며, 사회적 관계에서 어느 정도의 외로움을 느끼고 있다고 볼 수 있습니다.
> - 소속감에 대한 외로움은 사회적 역할 부족, 소속감 결여 등의 특징을 가지고 있으며, 사용자의 경우 건강 상태가 나빠 사회적 역할을 수행하기 어려워 소속감을 느끼기 어려운 상황으로 인해 나타납니다.
>   사용자의 소속감에 대한 외로움의 총합은 9점입니다. 따라서 이 수치가 낮은 외로움에 해당하며, 소속감에 대한 외로움은 비교적 적다고 볼 수 있습니다.
> 
> [대화 시스템 프롬프트]
> - 당신은 김영수 님을 위한 대화 상대가 되어주어야 합니다. 아래의 참고사항을 반영해 출력하세요.
> - 대화 중 사용자의 기억은 반드시 기억해야 합니다.
> - 말투는 친근하고 따뜻해야 할 것입니다.
> - 김영수 님의 건강 상태가 좋지 않다는 점을 고려하여, 건강에 대한 걱정과 지지를 표현해야 합니다.
> - 가족과의 관계에 대한 이야기를 나눌 때는 공감과 이해를 표현하며, 긍정적인 상호작용을 유도해야 합니다.
> - 사회적 활동에 대한 이야기를 나눌 때는 격려와 지지를 표현하며, 새로운 사회적 활동을 제안할 수 있습니다.
> 
> [분석 결과에 대한 이유]
> - 각 문항에 대한 외로움의 분석과 설명: 김영수 님의 답변을 바탕으로 가족관계, 사회적 관계, 소속감에 대한 외로움을 분석하였습니다. 가족관계 외로움은 가족과의 상호작용 부족과 정서적 지지 부족으로 나타났으며, 사회적 외로움은 사회적 지지 부족과 친구 및 이웃과의 상호작용 부족으로 나타났습니다. 소속감에 대한 외로움은 사회적 역할 부족과 소속감 결여로 나타났습니다.
> - 판단 근거: 김영수 님의 KGLS 점수와 각 문항에 대한 답변을 바탕으로 외로움의 정도를 판단하였습니다.
> - 정량적 지표: KGLS 점수와 각 하위요인(가족관계 외로움, 사회적 외로움, 소속감 결여)에 대한 점수를 통해 외로움의 정도를 정량적으로 평가하였습니다.